In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.format("json").load("inventory.json")

In [3]:
df.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- BuyCurrency: string (nullable = true)
 |-- BuyPrice: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- LeadTime: long (nullable = true)
 |-- MinOrderQuantity: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- SellCurrency: string (nullable = true)
 |-- SellPrice: double (nullable = true)
 |-- StockOnHand: long (nullable = true)
 |-- StockOnOrder: long (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- Supplier: string (nullable = true)
 |-- TargetBatchVolume: double (nullable = true)
 |-- Volume: double (nullable = true)



In [2]:
df.first()

Row(Brand=u'StarTAC', BuyCurrency=u'GBP', BuyPrice=27.67, Category=u'Technology', Id=16342939, LeadTime=15, MinOrderQuantity=1, Name=u'StarTAC Series', SellCurrency=u'USD', SellPrice=65.99, StockOnHand=15, StockOnOrder=50, SubCategory=u'Telephones and Communication', Supplier=u'Office First', TargetBatchVolume=0.0, Volume=0.8)

In [5]:
df.count()

100

In [4]:
from pyspark.sql.functions import count
df.select(count("Supplier")).show()

+---------------+
|count(Supplier)|
+---------------+
|            100|
+---------------+



In [6]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("Supplier")).show()

+------------------------+
|count(DISTINCT Supplier)|
+------------------------+
|                       6|
+------------------------+



In [7]:
from pyspark.sql.functions import min, max
df.select(min("StockOnHand"), max("StockOnHand")).show() 

+----------------+----------------+
|min(StockOnHand)|max(StockOnHand)|
+----------------+----------------+
|               0|              35|
+----------------+----------------+



In [8]:
from pyspark.sql.functions import sum
df.select(sum("StockOnHand")).show()

+----------------+
|sum(StockOnHand)|
+----------------+
|            1144|
+----------------+



In [12]:
df.groupBy("Supplier")\
.agg(sum("StockOnHand"))\
.show()

+------------+----------------+
|    Supplier|sum(StockOnHand)|
+------------+----------------+
|         RRD|             130|
|      Drecom|             449|
|     Logipro|             235|
|         FHL|             134|
| SuperSupply|              97|
|Office First|              99|
+------------+----------------+



In [20]:
from pyspark.sql.functions import sum, count, avg, expr
df.select(
    count("StockOnHand").alias("total_goods"),
    sum("StockOnHand").alias("total_units"),
    avg("StockOnHand").alias("avg_units"),
    expr("mean(StockOnHand)").alias("mean_units"))\
  .selectExpr("total_units/total_goods","avg_units","mean_units")\
  .show()

+---------------------------+---------+----------+
|(total_units / total_goods)|avg_units|mean_units|
+---------------------------+---------+----------+
|                      11.44|    11.44|     11.44|
+---------------------------+---------+----------+



In [25]:
df.select(
    expr("count(StockOnHand) AS total_goods"),
    expr("sum(StockOnHand) AS total_units"),
    expr("avg(StockOnHand) AS avg_units"),
    expr("mean(StockOnHand) AS mean_units"))\
  .selectExpr("total_units/total_goods","avg_units","mean_units")\
  .show()

+---------------------------+---------+----------+
|(total_units / total_goods)|avg_units|mean_units|
+---------------------------+---------+----------+
|                      11.44|    11.44|     11.44|
+---------------------------+---------+----------+



In [28]:
df.groupBy("Supplier").agg(
        count("StockOnHand").alias("quantity"),
        expr("count(StockOnHand)")).show()

+------------+--------+------------------+
|    Supplier|quantity|count(StockOnHand)|
+------------+--------+------------------+
|         RRD|      11|                11|
|      Drecom|      33|                33|
|     Logipro|      21|                21|
|         FHL|      14|                14|
| SuperSupply|      11|                11|
|Office First|      10|                10|
+------------+--------+------------------+



In [13]:
rolledUpDF = df.rollup("Category", "Supplier").agg(sum("StockOnHand"))\
    .withColumnRenamed("sum(StockOnHand)","total_units")\
    .select("Category", "Supplier", "total_units")\
    .orderBy("Category")
rolledUpDF.show()

+---------------+------------+-----------+
|       Category|    Supplier|total_units|
+---------------+------------+-----------+
|           null|        null|       1144|
|      Furniture|        null|        133|
|      Furniture|         RRD|         22|
|      Furniture|         FHL|         14|
|      Furniture|     Logipro|         24|
|      Furniture| SuperSupply|         36|
|      Furniture|      Drecom|         37|
|Office Supplies|         RRD|         48|
|Office Supplies|      Drecom|        345|
|Office Supplies| SuperSupply|         36|
|Office Supplies|     Logipro|        184|
|Office Supplies|        null|        734|
|Office Supplies|         FHL|        110|
|Office Supplies|Office First|         11|
|     Technology|         RRD|         60|
|     Technology|         FHL|         10|
|     Technology|      Drecom|         67|
|     Technology| SuperSupply|         25|
|     Technology|        null|        277|
|     Technology|Office First|         88|
+----------

In [15]:
rolledUpDF.where("Category IS NULL").show()

+--------+--------+-----------+
|Category|Supplier|total_units|
+--------+--------+-----------+
|    null|    null|       1144|
+--------+--------+-----------+



In [32]:
rolledUpDF.where("Supplier IS NULL").show()

+---------------+--------+-----------+
|       Category|Supplier|total_units|
+---------------+--------+-----------+
|           null|    null|       1144|
|      Furniture|    null|        133|
|Office Supplies|    null|        734|
|     Technology|    null|        277|
+---------------+--------+-----------+



In [16]:
df.cube("Category", "Supplier").agg(sum("StockOnHand"))\
    .withColumnRenamed("sum(StockOnHand)","total_units")\
    .select("Category", "Supplier", "total_units")\
    .orderBy("Supplier")\
    .show()

+---------------+------------+-----------+
|       Category|    Supplier|total_units|
+---------------+------------+-----------+
|      Furniture|        null|        133|
|Office Supplies|        null|        734|
|           null|        null|       1144|
|     Technology|        null|        277|
|           null|      Drecom|        449|
|     Technology|      Drecom|         67|
|      Furniture|      Drecom|         37|
|Office Supplies|      Drecom|        345|
|Office Supplies|         FHL|        110|
|     Technology|         FHL|         10|
|           null|         FHL|        134|
|      Furniture|         FHL|         14|
|     Technology|     Logipro|         27|
|Office Supplies|     Logipro|        184|
|      Furniture|     Logipro|         24|
|           null|     Logipro|        235|
|           null|Office First|         99|
|     Technology|Office First|         88|
|Office Supplies|Office First|         11|
|           null|         RRD|        130|
+----------